In [8]:
# data wangling
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# ML
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# custom modules
# add project src folder to sys.path so we can import preprocess.py
sys.path.append("../../src")
import preprocess

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# load data

train_df = pd.read_csv('../../data/raw/train.csv')
test_df = pd.read_csv('../../data/raw/test.csv')
submission_df = pd.read_csv('../../data/raw/gender_submission.csv')

X = train_df.drop("Survived", axis=1)
y = train_df["Survived"]
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
# Preprocess
X_processed = preprocess.preprocess(X)
test_processed = preprocess.preprocess(test_df)
X_processed.head()
# data leakage check
# ensure all the features are knowwn at boarding time

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,0,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,3,1,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,3,0,35.0,0,0,8.0500


In [11]:
def add_title_feature(df):
    df = df.copy()

    df["Title"] = df["Name"].str.extract(r",\s*([^\.]+)\.", expand=False)

    # Group rare titles
    df["Title"] = df["Title"].replace(
        ["Lady", "Countess", "Capt", "Col", "Don", "Dr",
         "Major", "Rev", "Sir", "Jonkheer", "Dona"],
        "Rare"
    )

    # Normalize similar titles
    df["Title"] = df["Title"].replace(
        {"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"}
    )

    return df["Title"]

X_processed["Title"] = add_title_feature(X)
test_processed["Title"] = add_title_feature(test_df)
X_processed.head()


,Pclass,Sex,Age,SibSp,Parch,Fare,Title
0,3,0,22.0,1,0,7.2500,Mr
1,1,1,38.0,1,0,71.2833,Mrs
2,3,1,26.0,0,0,7.9250,Miss
3,1,1,35.0,1,0,53.1000,Mrs
4,3,0,35.0,0,0,8.0500,Mr


In [12]:
X_processed = pd.get_dummies(X_processed, drop_first=True)
test_processed = pd.get_dummies(test_processed, drop_first=True)
X_processed, test_processed = X_processed.align(test_processed, join="left", axis=1, fill_value=0)
print("X_processed shape:", X_processed.shape)
print("test_processed shape:", test_processed.shape)

X_processed shape: (891, 11)
test_processed shape: (418, 11)


In [13]:
# -----------------------------
# Train / Validation split
# -----------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X_processed,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# -----------------------------
# Hyperparameter grid
# -----------------------------
param_grid = {
    "max_depth": [3, 4, 5, 6, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "criterion": ["gini", "entropy"]
}

# -----------------------------
# GridSearchCV setup
# -----------------------------
grid = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

# -----------------------------
# Run hyperparameter tuning
# -----------------------------
grid.fit(X_train, y_train)

print("Best Hyperparameters:", grid.best_params_)
print(f"Best CV Score: {grid.best_score_:.4f}")

# Best model from tuning
best_model = grid.best_estimator_

# -----------------------------
# Training accuracy
# -----------------------------
train_pred = best_model.predict(X_train)
train_acc = accuracy_score(y_train, train_pred)
print(f"Training Accuracy: {train_acc:.4f}")

# -----------------------------
# Validation accuracy
# -----------------------------
val_pred = best_model.predict(X_val)
val_acc = accuracy_score(y_val, val_pred)
print(f"Validation Accuracy: {val_acc:.4f}")

# -----------------------------
# Train final model on full data
# -----------------------------
model = DecisionTreeClassifier(
    **grid.best_params_,
    random_state=42
)
model.fit(X_processed, y)

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}
Best CV Score: 0.8147
Training Accuracy: 0.8624
Validation Accuracy: 0.8324


,criterion,'entropy'
,splitter,'best'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,4
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [14]:
# Predict test
test_pred = model.predict(test_processed)

# Create submission
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": test_pred
})
submission.to_csv("../../submissions/submission_exp006.csv", index=False)
print("submission_exp006.csv created")

submission_exp006.csv created
